# AI Services Example Notebook

This notebook demonstrates how to use the `ai_services.py` module from the WorkoutBuddy ML backend.

## Features Covered:
1. **Personalized Challenge Generation** - AI-powered daily workout challenges
2. **Community Compatibility Matching** - Find compatible workout partners
3. **Personalized Encouragement** - Generate motivational messages

## Prerequisites:
- Set up the `ANTHROPIC_API_KEY` in `.envrc` file (in project root)
- Install required dependencies from `requirements.txt`
- Have access to the database models

In [14]:
# Setup: Import modules and connect to the database
import os
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine


import sys
sys.path.append('../../src/ml_service')

from app.ai_services import ai_service
# Load environment variables (expects DATABASE_URL in .envrc or environment)
DATABASE_URL = os.getenv('DATABASE_URL', 'postgresql://wojciechkowalinski@localhost/workoutbuddy')
engine = create_engine(DATABASE_URL)
print(f'Connected to database: {DATABASE_URL}')

Anthropic API key not found. AI features will use fallback responses.
Connected to database: postgresql://wojciechkowalinski@localhost/workoutbuddy


In [15]:
# List all tables in the public schema
tables_query = """
SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' ORDER BY table_name;
"""
tables = pd.read_sql(tables_query, engine)
print('Available tables:')
display(tables)

# Preview the first few rows of each main table
main_tables = ['users', 'user_goals', 'user_stats', 'workouts', 'workout_exercises', 'exercises', 'friendships']
for table in main_tables:
    print(f'\n=== {table.upper()} ===')
    try:
        df = pd.read_sql(f'SELECT * FROM {table} LIMIT 5;', engine)
        display(df)
    except Exception as e:
        print(f'Could not read table {table}: {e}')

Available tables:


,table_name
0,exercises
1,friendships
2,user_goals
3,user_stats
4,users
5,workout_exercises
6,workouts



=== USERS ===


,id,email,username,hashed_password,full_name,is_active,is_verified,age,height,weight,fitness_goal,experience_level,created_at,updated_at



=== USER_GOALS ===


...

,id,workout_id,exercise_id,order,sets,reps,weight,duration,distance,speed,incline,rest_time,actual_reps,actual_weight,notes



=== EXERCISES ===


,id,name,description,primary_muscle,secondary_muscles,equipment,exercise_type,difficulty,instructions,tips,video_url,is_distance_based,is_time_based,mets
0,1,Barbell Back Squat,1) Stand with barbell on shoulders; 2) squat ...,LEGS,"[""Glutes""]",BARBELL,STRENGTH,3,"• Feet shoulder-width apart • Chest up, core t...",,ExRx.net/WeightExercises/Quadriceps/BBSquat,False,False,4.0
1,2,Conventional Deadlift,1) Lift barbell from floor to hip level with s...,LEGS,"[""Glutes"", ""Lower Back""]",BARBELL,STRENGTH,3,• Feet hip-width apart • Grip outside legs • C...,,ExRx.net/WeightExercises/ErectorSpinae/BBDeadlift,False,False,4.0
2,3,Sumo Deadlift,1) Wide stance deadlift with hands inside legs...,GLUTES,"[""Hamstrings"", ""Adductors""]",BARBELL,STRENGTH,3,"• Wide stance, toes out • Hands inside legs • ...",,YouTube: AthleanX Sumo Deadlift,False,False,4.0
3,4,Romanian Deadlift (Barbell),Hip hinge movement focusing on hamstring stretch,LEGS,"[""Glutes""]",BARBELL,FLEXIBILITY,3,,,,False,False,2.5
4,5,Barbell Bench Press,Lie on bench and press barbell from chest to a...,CHEST,[],OTHER,STRENGTH,3,• Feet flat on floor • Retract shoulder blades...,,ExRx.net/WeightExercises/PectoralSternal/BBBen...,False,False,4.0



=== FRIENDSHIPS ===


,id,user_id,friend_id,is_accepted,created_at,accepted_at


## 1. Setup Mock Data

Since we're running this notebook independently, we'll create mock user objects that match the database model structure.

In [16]:
# Create mock user class for testing
class MockUser:
    def __init__(self, user_id, goal_type="cardio", activity_level="moderate", 
                 year_of_birth=1990, first_name="John", last_name="Doe"):
        self.id = user_id
        self.goal_type = goal_type
        self.activity_level = activity_level
        self.year_of_birth = year_of_birth
        self.first_name = first_name
        self.last_name = last_name
        self.motivation_style = "encouraging"
        self.community_engagement_score = 0.7

class MockGoal:
    def __init__(self, goal_id, title, user_id):
        self.id = goal_id
        self.title = title
        self.owner_id = user_id

# Create sample users
user1 = MockUser(1, "cardio", "moderate", 1985, "Alice", "Johnson")
user2 = MockUser(2, "strength", "high", 1990, "Bob", "Smith")
user3 = MockUser(3, "flexibility", "low", 1995, "Carol", "Davis")
user4 = MockUser(4, "cardio", "moderate", 1988, "David", "Wilson")

# Create sample goals
goals_user1 = [MockGoal(1, "Run 5K in under 25 minutes", 1), MockGoal(2, "Improve cardiovascular health", 1)]
goals_user2 = [MockGoal(3, "Bench press bodyweight", 2), MockGoal(4, "Build muscle mass", 2)]
goals_user3 = [MockGoal(5, "Touch toes without bending knees", 3)]
goals_user4 = [MockGoal(6, "Complete a marathon", 4)]

print("Mock users and goals created successfully!")
print(f"User 1: {user1.first_name} {user1.last_name} - Goal: {user1.goal_type}, Activity: {user1.activity_level}")
print(f"User 2: {user2.first_name} {user2.last_name} - Goal: {user2.goal_type}, Activity: {user2.activity_level}")
print(f"User 3: {user3.first_name} {user3.last_name} - Goal: {user3.goal_type}, Activity: {user3.activity_level}")
print(f"User 4: {user4.first_name} {user4.last_name} - Goal: {user4.goal_type}, Activity: {user4.activity_level}")

Mock users and goals created successfully!
User 1: Alice Johnson - Goal: cardio, Activity: moderate
User 2: Bob Smith - Goal: strength, Activity: high
User 3: Carol Davis - Goal: flexibility, Activity: low
User 4: David Wilson - Goal: cardio, Activity: moderate


## 2. Personalized Challenge Generation

Generate AI-powered personalized daily challenges based on user profile and history.

In [17]:
# Example user history and preferences
user_history = {
    "recent_completion_rate": "85% - Very consistent",
    "recent_challenges": [
        "20-minute HIIT workout",
        "Morning jog routine",
        "Bodyweight circuit"
    ]
}

user_preferences = {
    "equipment": "Dumbbells",
    "time_minutes": 20,
    "preferred_intensity": "moderate"
}

# Generate personalized challenge
async def demo_challenge_generation(user):
    print("\n=== PERSONALIZED CHALLENGE GENERATION ===")
    print(f"Generating challenge for: {user.first_name} {user.last_name}")
    print(f"Goal Type: {user.goal_type}")
    print(f"Activity Level: {user.activity_level}")
    print(f"Age: {datetime.now().year - user.year_of_birth}")
    
    challenge = await ai_service.generate_personalized_challenge(
        user=user,
        user_history=user_history,
        preferences=user_preferences
    )
    
    print("\n📋 Generated Challenge:")
    print(f"Title: {challenge.title}")
    print(f"Description: {challenge.description}")
    print(f"Duration: {challenge.duration}")
    print(f"Difficulty: {challenge.difficulty}/5")
    print(f"Equipment Needed: {challenge.equipment_needed}")
    print(f"Motivation: {challenge.motivation_message}")
    print(f"AI Generated: {challenge.ai_generated}")
    
    return challenge

# Run the challenge generation
challenge_result = await demo_challenge_generation(user1)



=== PERSONALIZED CHALLENGE GENERATION ===
Generating challenge for: Alice Johnson
Goal Type: cardio
Activity Level: moderate
Age: 40

📋 Generated Challenge:
Title: Morning Mobility
Description: Gentle flow: neck rolls, shoulder circles, hip circles, calf raises. Hold each for 30 seconds.
Duration: 8 minutes
Difficulty: 1/5
Equipment Needed: []
Motivation: Your body will thank you for this care! 🧘‍♀️
AI Generated: False


In [18]:
async def test_different_user_challenges():
    users_to_test = [user2, user3]  # Strength and flexibility users
    
    for user in users_to_test:
        print(f"\n\n=== CHALLENGE FOR {user.first_name.upper()} ===")
        print(f"Goal: {user.goal_type} | Activity Level: {user.activity_level}")
        
        # Adjust history based on user type
        adapted_history = {
            "recent_completion_rate": "70% - Good progress",
            "recent_challenges": [
                f"{user.goal_type.title()} focused workout",
                "Beginner routine"
            ]
        }
        
        challenge = await ai_service.generate_personalized_challenge(
            user=user,
            user_history=adapted_history,
            preferences={"time_minutes": 15}
        )
        
        print(f"📋 {challenge.title}")
        print(f"📝 {challenge.description}")
        print(f"⏱️ Duration: {challenge.duration}")
        print(f"💪 Motivation: {challenge.motivation_message}")

await test_different_user_challenges()



=== CHALLENGE FOR BOB ===
Goal: strength | Activity Level: high
📋 Morning Mobility
📝 Gentle flow: neck rolls, shoulder circles, hip circles, calf raises. Hold each for 30 seconds.
⏱️ Duration: 8 minutes
💪 Motivation: Your body will thank you for this care! 🧘‍♀️


=== CHALLENGE FOR CAROL ===
Goal: flexibility | Activity Level: low
📋 Morning Mobility
📝 Gentle flow: neck rolls, shoulder circles, hip circles, calf raises. Hold each for 30 seconds.
⏱️ Duration: 8 minutes
💪 Motivation: Your body will thank you for this care! 🧘‍♀️


## 3. Community Compatibility Matching

Demonstrate AI-powered matching between users for accountability partnerships.

In [19]:
async def demo_community_matching(user):
    print("\n\n=== COMMUNITY COMPATIBILITY MATCHING ===")
    print(f"Finding matches for: {user1.first_name} {user1.last_name}")
    print(f"Looking for accountability partners...")
    
    # Potential matches (excluding the user themselves)
    potential_matches = [user2, user3, user4]
    
    # Goals for each user
    match_goals = {
        user2.id: goals_user2,
        user3.id: goals_user3,
        user4.id: goals_user4
    }
    
    matches = await ai_service.analyze_community_compatibility(
        user=user,
        potential_matches=potential_matches,
        user_goals=goals_user1,
        match_goals=match_goals
    )
    
    print(f"\n🤝 Found {len(matches)} potential matches:")
    print("=" * 50)
    
    for i, match in enumerate(matches, 1):
        print(f"\n#{i} Match: {match.name}")
        print(f"🎯 Compatibility Score: {match.compatibility_score:.1%}")
        print(f"💡 Why they're a good match:")
        for reason in match.match_reasons:
            print(f"   • {reason}")
        print(f"🏃 Shared Interests: {', '.join(match.shared_interests)}")
    
    return matches

matches_result = await demo_community_matching(user1)




=== COMMUNITY COMPATIBILITY MATCHING ===
Finding matches for: Alice Johnson
Looking for accountability partners...

🤝 Found 3 potential matches:

#1 Match: User 2
🎯 Compatibility Score: 75.8%
💡 Why they're a good match:
   • Similar fitness goals and experience level
   • Compatible workout schedules
   • Shared motivation style
🏃 Shared Interests: fitness, health, wellness

#2 Match: User 3
🎯 Compatibility Score: 86.7%
💡 Why they're a good match:
   • Similar fitness goals and experience level
   • Compatible workout schedules
   • Shared motivation style
🏃 Shared Interests: fitness, health, wellness

#3 Match: User 4
🎯 Compatibility Score: 80.6%
💡 Why they're a good match:
   • Similar fitness goals and experience level
   • Compatible workout schedules
   • Shared motivation style
🏃 Shared Interests: fitness, health, wellness


## 4. Personalized Encouragement Generation

Generate contextual encouragement messages based on user progress and sentiment.

In [20]:
async def demo_encouragement_generation():
    print("\n\n=== PERSONALIZED ENCOURAGEMENT GENERATION ===")
    
    # Different user contexts to test
    test_scenarios = [
        {
            "name": "High Performer",
            "context": {
                "recent_checkin": "Completed 5K run in 24:30 - new personal record!",
                "progress_trend": "Improving consistently",
                "engagement_level": "High",
                "days_since_last": 1,
                "recent_challenges_completed": 6
            }
        },
        {
            "name": "Struggling User",
            "context": {
                "recent_checkin": "Missed workouts this week, feeling unmotivated",
                "progress_trend": "Declining",
                "engagement_level": "Low",
                "days_since_last": 7,
                "recent_challenges_completed": 1
            }
        },
        {
            "name": "Comeback User",
            "context": {
                "recent_checkin": "Back to working out after a break",
                "progress_trend": "Stable",
                "engagement_level": "Medium",
                "days_since_last": 3,
                "recent_challenges_completed": 2
            }
        }
    ]
    
    for scenario in test_scenarios:
        print(f"\n📱 Scenario: {scenario['name']}")
        print(f"Context: {scenario['context']['recent_checkin']}")
        
        encouragement = await ai_service.generate_encouragement(
        user=user,
            context=scenario['context']
        )
        
        print(f"\n💬 Generated Encouragement:")
        print(f"Message: \"{encouragement.message}\"")
        print(f"Tone: {encouragement.tone}")
        print(f"Personalized: {encouragement.personalized}")
        print(f"Suggestions:")
        for suggestion in encouragement.suggestions:
            print(f"   • {suggestion}")
        print("-" * 50)

await demo_encouragement_generation()



=== PERSONALIZED ENCOURAGEMENT GENERATION ===

📱 Scenario: High Performer
Context: Completed 5K run in 24:30 - new personal record!


NameError: name 'user' is not defined

## 5. Fallback Behavior Testing

Test how the service behaves when AI is unavailable (without API key).

In [20]:
async def demo_fallback_behavior():
    print("\n\n=== FALLBACK BEHAVIOR TESTING ===")
    print("Testing AI service without API key (fallback mode)")
    
    # Create a service instance without API key
    from app.ai_services import AIService
    
    # Temporarily remove API key
    original_key = os.environ.get('ANTHROPIC_API_KEY')
    if 'ANTHROPIC_API_KEY' in os.environ:
        del os.environ['ANTHROPIC_API_KEY']
    
    fallback_service = AIService()
    
    print(f"AI Service enabled: {fallback_service.enabled}")
    
    # Test fallback challenge generation
    print("\n📋 Fallback Challenge Generation:")
    fallback_challenge = await fallback_service.generate_personalized_challenge(
        user=user2,  # strength user
        user_history=user_history
    )
    
    print(f"Title: {fallback_challenge.title}")
    print(f"Description: {fallback_challenge.description}")
    print(f"AI Generated: {fallback_challenge.ai_generated}")
    
    # Test fallback encouragement
    print("\n💬 Fallback Encouragement:")
    fallback_encouragement = await fallback_service.generate_encouragement(
        user=user,
        context={"progress_trend": "stable"}
    )
    
    print(f"Message: \"{fallback_encouragement.message}\"")
    print(f"Personalized: {fallback_encouragement.personalized}")
    
    # Restore API key
    if original_key:
        os.environ['ANTHROPIC_API_KEY'] = original_key
    
    print("\n✅ Fallback testing completed!")

await demo_fallback_behavior()

Anthropic API key not found. AI features will use fallback responses.




=== FALLBACK BEHAVIOR TESTING ===
Testing AI service without API key (fallback mode)
AI Service enabled: False

📋 Fallback Challenge Generation:
Title: Bodyweight Power
Description: 3 rounds: 8 push-ups, 12 squats, 15-second plank. Rest 1 minute between rounds.
AI Generated: False

💬 Fallback Encouragement:
Message: "Your consistency is inspiring! Keep it up! ✨"
Personalized: False

✅ Fallback testing completed!


## 6. Usage Instructions

### Environment Setup
```bash
# From project root, make sure .envrc file contains your API key:
# export ANTHROPIC_API_KEY="your-api-key-here"

# If using direnv (recommended):
direnv allow

# Or manually source the file:
source .envrc

# Install dependencies
pip install -r ml_backend/requirements.txt

# Navigate to notebook directory and start Jupyter
cd docs/analytics/notebooks
jupyter notebook ai_services_example.ipynb
```

### Production Considerations
1. **Rate Limiting**: Implement rate limiting to avoid API quota issues
2. **Caching**: Cache similar requests to reduce costs
3. **Error Handling**: Always have fallback responses ready
4. **Monitoring**: Track API usage and costs
5. **User Consent**: Ensure users consent to AI-generated content
6. **Security**: Never commit API keys to version control

### Integration Example
```python
from app.ai_services import ai_service

@app.post("/generate-challenge")
async def generate_challenge(user_id: int, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.id == user_id).first()
    challenge = await ai_service.generate_personalized_challenge(
        user=user,
        user_history=get_user_history(user_id),
        preferences=get_user_preferences(user_id)
    )
    return challenge
```

In [21]:
print("\n\n🎉 AI Services Demo Complete!")
print("\nKey Features Demonstrated:")
print("✅ Personalized Challenge Generation")
print("✅ Community Compatibility Matching")
print("✅ Personalized Encouragement")
print("✅ Fallback Behavior")
print("\nThe AI service is ready for integration into your WorkoutBuddy application!")



🎉 AI Services Demo Complete!

Key Features Demonstrated:
✅ Personalized Challenge Generation
✅ Community Compatibility Matching
✅ Personalized Encouragement
✅ Fallback Behavior

The AI service is ready for integration into your WorkoutBuddy application!
